### Install required libraries

In [1]:
!pip install  "tensorflow-text>=2.11"
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00


### Import library

In [2]:
import numpy as np

import typing
from typing import Tuple, Any

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text

### Import the data

In [3]:
import pathlib
file_path = "/content/data for DA to MSA.txt"
path_to_file = pathlib.Path(file_path)
path_to_file

PosixPath('/content/data for DA to MSA.txt')

In [6]:
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  context = np.array([context for context, target in pairs])
  target = np.array([target for context, target in pairs])

  return target, context

In [7]:
target_raw, context_raw = load_data(path_to_file)
print(context_raw[-1])
print(target_raw[-1])

خمسة وعشرين دولار مع رسوم الباص، والمرشد ورسوم التسجيل.
خمسة وعشرين دولاراً شاملة أجرة الحافلة ، ومصاريف المرشد والدخول .


In [6]:
BUFFER_SIZE = len(target_raw)
BATCH_SIZE = 64

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (tf.data.Dataset
             .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
             .shuffle(BUFFER_SIZE)
             .batch(BATCH_SIZE)
             )
val_raw = (tf.data.Dataset
             .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
             .shuffle(BUFFER_SIZE)
             .batch(BATCH_SIZE)
             )


In [7]:
for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_target_strings[:5])
  print()
  print(example_context_strings[:5])

tf.Tensor(
[b'\xd8\xa3\xd8\xb1\xd9\x8a\xd8\xaf \xd8\xa3\xd9\x86 \xd8\xa3\xd9\x84\xd8\xaa\xd8\xad\xd9\x82 \xd8\xa8\xd8\xac\xd9\x88\xd9\x84\xd8\xa9 \xd8\xb4\xd9\x84\xd8\xa7\xd9\x84\xd8\xa7\xd8\xaa \xd9\x86\xd9\x8a\xd8\xa7\xd8\xac\xd8\xb1\xd8\xa7 .'
 b' \xd9\x85\xd9\x86 \xd9\x81\xd8\xb6\xd9\x84\xd9\x83 \xd8\xa7\xd8\xb0\xd9\x87\xd8\xa8 \xd8\xb9\xd9\x84\xd9\x89 \xd9\x82\xd8\xa7\xd8\xb9\xd8\xa9 \xd8\xa7\xd9\x84\xd9\x85\xd8\xba\xd8\xa7\xd8\xaf\xd8\xb1\xd8\xa9 \xd8\xa7\xd9\x84\xd8\xaf\xd9\x88\xd9\x84\xd9\x8a\xd8\xa9 , \xd8\xa3\xd8\xaa\xd9\x85\xd9\x86\xd9\x89 \xd9\x84\xd9\x83 \xd8\xb1\xd8\xad\xd9\x84\xd8\xa9 \xd8\xb3\xd8\xb9\xd9\x8a\xd8\xaf\xd8\xa9 .'
 b'\xd8\xa3\xd9\x8a\xd9\x86 \xd8\xb7\xd8\xa7\xd9\x88\xd9\x84\xd8\xa9 \xd8\xa7\xd9\x84\xd8\xaa\xd8\xb3\xd8\xac\xd9\x8a\xd9\x84 \xd8\xa7\xd9\x84\xd8\xae\xd8\xa7\xd8\xb5\xd8\xa9 \xd8\xa8\xd8\xb4\xd8\xb1\xd9\x83\xd8\xa9 \xd8\xaf\xd9\x84\xd8\xaa\xd8\xa7 \xd8\x9f'
 b'\xd8\xab\xd9\x85\xd8\xa9 \xd8\xb9\xd8\xb7\xd9\x84 \xd9\x85\xd8\xa7 \xd8\xa8\xd8\xa7\xd9

In [8]:
def tf_lower_and_split_punct(text):
  text = tf_text.normalize_utf8(text,'NFKD')
  text = tf.strings.lower(text)
  #replace all char expecpt regex below with spaces
  text = tf.strings.regex_replace(text,'[^a-z.,?!¿]','')
  #put space for after each string included in reg
  text = tf.strings.regex_replace(text,'[a-z.,?!¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text


In [16]:
example_text = tf.constant('كيف أخبارك وبتعمل ايه دلوقتى')

In [22]:
example_text.numpy().decode()

'كيف أخبارك وبتعمل ايه دلوقتى'

In [23]:
tf_lower_and_split_punct(example_text).numpy().decode()

'[START]  [END]'

In [12]:
max_vocab_size = 5000
context_text_processor = tf.keras.layers.TextVectorization(
    standardize= tf_lower_and_split_punct,
    max_tokens = max_vocab_size,
    ragged=True
)

In [13]:
# adapt the input data for specification at textVectorization layer
context_text_processor.adapt(train_raw.map(lambda context,target:context))

In [14]:
example_tokens = context_text_processor(example_context_strings)
example_tokens[:3,:]

<tf.RaggedTensor [[2, 4, 3], [2, 4, 4, 3], [2, 3]]>

 convert token id into text again using get_vocabulary method

In [15]:
context_vocab = np.array(context_text_processor.get_vocabulary())
tokens = context_vocab[example_tokens[0].numpy()]
' '.join(tokens)

'[START] . [END]'